# Imports

In [1]:
import sys
 
# setting path
sys.path.append('..')

In [2]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee

# Map

In [3]:
Map = geemap.Map(center=[-11.8673, -76.9544], zoom=2.5)
Map

Map(center=[-11.8673, -76.9544], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# Dataset builder

In [4]:
ee_dataset_builder = EEDatasetBuilder()

Make the biomass band with some filters

In [5]:
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested', #'Spawn_AGB_tCO2e',  #'Deforestation_risk_response_variable_brazil',
    filter_dict={
                # 'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 # 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)

Make covariates bands

In [6]:
ee_dataset_builder.spatial_covariates(covariates=['brazil_roads', 'terrain', 'brazil_protected_areas', 'population_density',
                                                 'forest_age', 'urbanization', 'brazil_surrounding_forest', 'brazil_pasture',
                                                 'brazil_agriculture', 'south_america_rivers', 'urban_distance']) # TODO: 'urban_distance'

List band names of image

In [7]:
ee_dataset_builder.image.bandNames().getInfo()

['Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested',
 'brazil_roads',
 'elevation',
 'aspect',
 'slope',
 'hillshade',
 'brazil_protected_areas',
 'population_density',
 'forest_age',
 'urbanization',
 'brazil_surrounding_forest',
 'brazil_pasture',
 'brazil_agriculture',
 'south_america_rivers',
 'urban_distance']

So the image has the added bands; it basically seems to work.

Use the below code to make a plot and confirm results of mask match what is expected for both biomass and the features.

### Visualize in map

In [8]:
biomass_vis_param = { 
  'min': 0.0,
  'max': 1.0,
  'palette': ['green','red']}

Map.addLayer(ee_dataset_builder.image.select('Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested'), biomass_vis_param, 
             'Deforestation_risk_response_variable_brazil', True)

In [9]:
roads_vis_param = { 
  'min': 0.0,
  'max': 10000.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};

Map.addLayer(ee_dataset_builder.image.select('brazil_roads'), roads_vis_param, 'Roads distance', False)
Map.addLayer(ee_dataset_builder.image.select('south_america_rivers'), roads_vis_param, 'Rivers distance', False)

In [10]:
protected_areas_vis_param = { 
  'min': 0.0,
  'max': 200000.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_protected_areas'), protected_areas_vis_param, 'Distance to Protected Areas', False)

In [11]:
population_density_vis_param = { 
  'min': 0.0,
  'max': 500.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};
Map.addLayer(ee_dataset_builder.image.select('population_density'), population_density_vis_param, 'Population Density', False)

In [12]:
forest_age_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ]};
Map.addLayer(ee_dataset_builder.image.select('forest_age'), forest_age_vis_param, 'Forest age', False)

In [13]:
urbanization_vis_param = { 
  'min': 0.0,
  'max': 7.0,
  'palette':  [
    'green','yellow','orange','red'
  ]};
Map.addLayer(ee_dataset_builder.image.select('urbanization'), urbanization_vis_param, 'Urbanization', False)

In [14]:
surrounding_forest_vis_param = { 
  'min': 0.0,
  'max': 700.0,
  'palette':  [
    'green','yellow','orange','red'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_surrounding_forest'), surrounding_forest_vis_param, 'Surrounding Forest', False)

In [15]:
pasture_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_pasture'), pasture_vis_param, 'Surrounding Pasture', False)

In [16]:
agriculture_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ],
    'format': 'png'};
Map.addLayer(ee_dataset_builder.image.select('brazil_agriculture'), agriculture_vis_param, 'Surrounding Agriculture', False)

In [17]:
urban_distance_vis_param = { 
  'min': 0.0,
  'max': 10000.0,
  'palette':  [
    'red','green'
  ]};
Map.addLayer(ee_dataset_builder.image.select('urban_distance'), urban_distance_vis_param, 'Distance to Urban Areas', False)

## Test and val sets

In [18]:
ee_dataset_builder.test_set(feature_collection='users/margauxmf-earthshot/test_set_Atlantic_Forest_deforestation_risk',
                            buffer=5000, test_set_name='test_set_5km_buffer')
ee_dataset_builder.test_set(feature_collection='users/margauxmf-earthshot/test_set_Atlantic_Forest_deforestation_risk',
                            buffer=None, test_set_name='test_set_no_buffer')
ee_dataset_builder.test_set(feature_collection='users/margauxmf-earthshot/validation_set_Atlantic_Forest_deforestation_risk',
                            buffer=5000, test_set_name='val_set_5km_buffer')
ee_dataset_builder.test_set(feature_collection='users/margauxmf-earthshot/validation_set_Atlantic_Forest_deforestation_risk',
                            buffer=None, test_set_name='val_set_no_buffer')

In [19]:
vis = { 
  'min': 0.0,
  'max': 2.0,
  'palette': ['d41c63', '33ffcc']}
Map.addLayer(ee_dataset_builder.image.select('test_set_5km_buffer'),
             vis,
             'test_set_5km_buffer', False)
Map.addLayer(ee_dataset_builder.image.select('test_set_no_buffer'),
             vis,
             'test_set_no_buffer', False)
Map

Map(bottom=1392.0, center=[-11.8673, -76.9544], controls=(WidgetControl(options=['position', 'transparent_bg']…

In [20]:
ee_dataset_builder.image.bandNames().getInfo()

['Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested',
 'brazil_roads',
 'elevation',
 'aspect',
 'slope',
 'hillshade',
 'brazil_protected_areas',
 'population_density',
 'forest_age',
 'urbanization',
 'brazil_surrounding_forest',
 'brazil_pasture',
 'brazil_agriculture',
 'south_america_rivers',
 'urban_distance',
 'test_set_5km_buffer',
 'test_set_no_buffer',
 'val_set_5km_buffer',
 'val_set_no_buffer']

# Export samples CSV to GCP

Check progress in your GEE console: https://code.earthengine.google.com/tasks

In [21]:
# Gridded world shapefile asset in GEE
shp_asset_path = 'projects/ee-earthshot-shapefiles/assets/Altantic_Forest_10km2_grid'  #Atlantic_Forest_100km2_grid'

gcp_bucket = 'eartshot-science-team'
gcp_folder_name = 'deforestation_risk'
scale = 30
numPoints = 1000 # we override the number of points below with classPoints
classBand = "Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested"
classPoints = [8500, 1500]
classValues = [0,1]

In [ ]:
samples_folder_name = f'Brazil_Atlantic_Forest_10km2_grids_samples_csv_scale{scale}_{numPoints}numPoints'

# Stratified Sampling
# This will take quite some time
ee_dataset_builder.samples_csv_export(shp_asset_path=shp_asset_path, 
                                      name_gcp_bucket=gcp_bucket, 
                                      folder_in_gcp_bucket=gcp_folder_name + '/' + samples_folder_name, 
                                      scale=scale, 
                                      geometries=True,
                                      isStratifiedSampling=True, 
                                      numPoints=numPoints, 
                                      classValues=classValues,
                                      classBand=classBand, 
                                      classPoints=classPoints)

Geometry number of features: 17808
Stratified sampling: 
numPoints: 1000, 
classBand: Response_Variable_Brazil_Atlantic_Forest_0forest_1deforested, 
scale: 30, 
geometries: True, 
dropNulls: True, 
tileScale: 1, 
classPoints: [8500, 1500], 
seed: 0, 
projection:None


  0%|                                                                                                                                                    | 6/17808 [00:01<52:43,  5.63it/s]

# Export tiles to GCP

In [20]:
ee_dataset_builder_inference = EEDatasetBuilder()
ee_dataset_builder_inference.filtered_response_layer_from_raster(
    response_raster = None,
    filter_dict={
        'forest_Brazil_Atlantic_Forest_non_Forest': {}
    }
)
# Only building predictors here
ee_dataset_builder_inference.spatial_covariates(covariates=['brazil_roads', 'terrain', 'brazil_protected_areas', 'population_density',
                                                 'forest_age', 'urbanization', 'brazil_surrounding_forest', 'brazil_pasture',
                                                 'brazil_agriculture', 'south_america_rivers', 'urban_distance']) 

In [21]:
ee_dataset_builder_inference.image.bandNames().getInfo()

['brazil_roads',
 'elevation',
 'aspect',
 'slope',
 'hillshade',
 'brazil_protected_areas',
 'population_density',
 'forest_age',
 'urbanization',
 'brazil_surrounding_forest',
 'brazil_pasture',
 'brazil_agriculture',
 'south_america_rivers',
 'urban_distance']

In [22]:
forest_age_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ]};
Map.addLayer(ee_dataset_builder_inference.image.select('forest_age'), forest_age_vis_param, 'Inference Forest age')

In [25]:
################## Export inference tiles to GCP bucket ##################
# shp_asset_path_tiles = 'projects/ee-earthshot-shapefiles/assets/limite_MA2_grid_2degrees'
shp_asset_path_tiles = 'projects/ee-earthshot-shapefiles/assets/Altantic_Forest_200km2_grid'

tiles_folder_name = f'Brazil_Deforestation_Risk_inference_200km2_grid_scale{scale}'
maxPixels = 1e13

print(f'\nExport inference tiles using the shapefile: {shp_asset_path_tiles}...')
# This will take quite some time
ee_dataset_builder_inference.tiles_export(shp_asset_path_tiles,
                                    name_gcp_bucket=gcp_bucket,
                                    folder_in_gcp_bucket=gcp_folder_name + '/' + tiles_folder_name,
                                    maxPixels=maxPixels, scale=scale)
print('Inference tiles export done.\n')


Export inference tiles using the shapefile: projects/ee-earthshot-shapefiles/assets/limite_MA2_grid_2degrees...
Geometry number of features: 58

Starting collecting tiles...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58/58 [00:17<00:00,  3.37it/s]

Inference tiles export done.

